<a href="https://colab.research.google.com/github/jalva80/NLPlaying/blob/master/NER_polyglot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reconnaissance d'Entités Nommées sur un texte rédigé en langue française à l'aide de la librairie polyglot.
https://polyglot.readthedocs.io/en/latest/index.html


Téléchargement des librairies non disponibles sur Google Colaboratory

In [1]:
!pip install polyglot
!pip install PyICU
!pip install pycld2
!pip install morfessor

     |████████████████████████████████| 133kB 2.9MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52557 sha256=6bd70e095d15d180e3d6c9b7d1b4afd55cf758f374ad3e24b1b687b96310cd8f
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
Successfully built polyglot
     |████████████████████████████████| 235kB 2.8MB/s 
  Created wheel for PyICU: filename=PyICU-2.5-cp36-cp36m-linux_x86_64.whl size=1252601 sha256=4ddd15bd152f796f5fd542137731a5fc3ff757048ffa9f32f44611f732a9142a
  Stored in directory: /root/.cache/pip/wheels/07/3a/28/09f90c38785945ddf9af61b7add1aa62a740f40e259626ef3a
Successfully built PyICU
     |████████████████████████████████| 41.4MB 100kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833562 sha256=4e8b4b30fd3f949470ea380cf3ca353da6292b4c43d7ba64806238b293f24bc8
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3d

Téléchargement des éléments du modèle français.

In [2]:
!polyglot download embeddings2.fr ner2.fr

[polyglot_data] Downloading package embeddings2.fr to
[polyglot_data]     /root/polyglot_data...
[polyglot_data] Downloading package ner2.fr to /root/polyglot_data...


In [0]:
from polyglot.text import Text
from bs4 import BeautifulSoup
import requests
import re

Définition d'une fonction url_to_string qui récupère le contenu d'une page Web à partir de son url:

In [0]:
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

J'effectue des essais sur le texte du Comte de Monte Cristo (A. Dumas). 
Le site où je récupère le texte ne comporte pas de balises pour délimiter la partie qui m'intéresse, donc j'utilise des champs spécifiques pour ne conserver que le texte du livre.

In [0]:
url = "http://textes.libres.free.fr/francais/alexandre-dumas-pere_le-comte-de-monte-cristo-tome-i.htm"

page_content = url_to_string(url)

start_phrase = r'TEXTE INTÉGRAL'
end_phrase = r'FIN DU TOME'
start = next(re.finditer(start_phrase, page_content)).end() + 1
end = next(re.finditer(end_phrase, page_content)).start()

In [0]:
texte = page_content[start:end]

On utilise donc la librairie polyglot pour identifier les entités, et plus précisément la classe Text.
On commence par imprimer les entités trouvées dans le texte:

In [8]:
# Create a new text object using Polyglot's Text class: txt
txt = Text(texte)

# Print each of the entities found
for ent in txt.entities[10:20]:
    print(ent)

['Jaros']
['beaupré']
['beaupré', 'décrochés']
['Marseille']
['Saint', '-', 'Jean']
['Réserve']
['Dantès']
['Morrel', '!']
['Civita', '-', 'Vecchia']
['Leclère']


In [0]:
# Print the type of ent
print(type(ent))

<class 'polyglot.text.Chunk'>


Je crée une liste de tuples de la forme( type d'entité, entité du texte)

In [14]:
# Create the list of tuples: entities
entities = [(ent.tag, ' '.join(ent)) for ent in txt.entities]

# Print entities
print(entities[10:20])

[('I-LOC', 'Jaros'), ('I-PER', 'beaupré'), ('I-LOC', 'beaupré décrochés'), ('I-LOC', 'Marseille'), ('I-LOC', 'Saint - Jean'), ('I-ORG', 'Réserve'), ('I-PER', 'Dantès'), ('I-PER', 'Morrel !'), ('I-LOC', 'Civita - Vecchia'), ('I-PER', 'Leclère')]


On trouve des entités de Lieu (Marseille, Saint-Jean), ainsi que des personnes (Dantès, Morrel)